The Bramble-Pasciak Transformation
===

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.05)).Curve(3)

         version v6.2.2105-181-ga2f7b918 but
         version v6.2.2105-201-g376fe7c6 is loaded at run-time!!!


In [2]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = L2(mesh, order=1)

u,v = V.TnT()
p,q = Q.TnT()

bfa = BilinearForm(InnerProduct(grad(u),grad(v))*dx).Assemble()
bfb = BilinearForm(div(u)*q*dx).Assemble()

gfu = GridFunction(V)
gfp = GridFunction(Q)

uin = CF( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

inva = bfa.mat.Inverse(freedofs=V.FreeDofs())
# gfu.vec.data += inv * res


bfschur = BilinearForm(p*q*dx, diagonal=True).Assemble()
preschur = bfschur.mat.Inverse()

Draw(gfu, mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-264-g4a22558b6', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

The saddle point system
$$
\left( \begin{array}{ccc} A & B^T \\ B & 0 \end{array} \right)
\left( \begin{array}{c} u \\ p  \end{array} \right)
=
\left( \begin{array}{c} f \\ g  \end{array} \right)
$$
is multiplied from the left by the transformation matrices
$$
\left( \begin{array}{ccc} A-\hat A & 0 \\ 0 & I \end{array} \right)
\left( \begin{array}{ccc} \hat I & 0 \\ B & -I \end{array} \right)
\left( \begin{array}{ccc} \hat A^{-1} & 0 \\ 0 & I \end{array} \right)
$$

to obtain the transformed system

$$
\left( \begin{array}{ccc} (A-\hat A) \hat A^{-1} A & (A-\hat A) \hat A^{-1} B^T \\ 
 B \hat A^{-1} (A - \hat A) & B \hat A^{-1} B^T \end{array} \right)
\left( \begin{array}{c} u \\ p  \end{array} \right)
=
\left( \begin{array}{c} (A-\hat A) A^{-1} f \\ B \hat A^{-1} f -g  \end{array} \right)
$$


In [3]:
def BramblePasciakCG(A, B, f, g, preA, preS):
    preA = 1.2*preA   # scaling
    maxit = 50
    tol = 1e-8
    
    xu = f.CreateVector()
    wu = f.CreateVector()
    ru = f.CreateVector() 
    pru = f.CreateVector() 
    pu = f.CreateVector()
    apu = f.CreateVector()

    xp = g.CreateVector()
    wp = g.CreateVector()
    rp = g.CreateVector()
    pp = g.CreateVector()
    app = g.CreateVector()

    # r.data = b
    # p.data = pre*r
    # wrn = InnerProduct(r,p)
    # err0 = sqrt(wrn)
    # x[:] = 0

    pu.data = preA * f
    pru.data = pu
    ru.data = A*pu-f
    rp.data = B * pu - g
    pp.data = preS * rp
    
    wrn = InnerProduct(pu, ru) + InnerProduct(rp,pp)
    err0 = sqrt(wrn)
    
    x = BlockVector([xu,xp])
    p = BlockVector([pu,pp])
    w = BlockVector([wu,wp])
    r = BlockVector([ru,rp])
    ap = BlockVector([apu,app])

    
    x[:] = 0
    for it in range(maxit):
        # ap.data = A * p
        hv = (A * p[0] + B.T * p[1]).Evaluate()
        papu = preA * hv
        ap[0].data = A * papu - hv
        ap[1].data = B * (papu - pu)
        
        pap = InnerProduct(p, ap)
        wr = wrn
        alpha = wr / pap
        
        x.data += alpha * p
        r.data -= alpha * ap
        pru.data -= alpha * papu

        # w.data = pre*r
        wu.data = pru
        wp.data = preS * rp
        
        wrn = InnerProduct(w, r)
        err = sqrt(wrn)
        print ("Iteration",it,"err=",err)
        if err < tol * err0: break
            
        beta = wrn / wr
        
        p *= beta
        p.data += w 
    
    return xu,xp

In [4]:
resf = (-bfa.mat * gfu.vec).Evaluate()
resg = (-bfb.mat * gfu.vec).Evaluate()

sol = BramblePasciakCG (bfa.mat, bfb.mat, resf, resg, inva, preschur)

gfu.vec.data += sol[0]
gfp.vec.data += sol[1]

Draw (gfu);

Iteration 0 err= 2.269429957278514
Iteration 1 err= 2.4551399765458344
Iteration 2 err= 2.560479023395072
Iteration 3 err= 1.833076951883552
Iteration 4 err= 1.6527754302249622
Iteration 5 err= 1.1325138713578506
Iteration 6 err= 1.5393723288052816
Iteration 7 err= 1.5781427283147023
Iteration 8 err= 1.3386048627193023
Iteration 9 err= 1.6021296117029227
Iteration 10 err= 1.2046885104258616
Iteration 11 err= 1.3079076197644617
Iteration 12 err= 0.8023021620236153
Iteration 13 err= 0.7167713284917975
Iteration 14 err= 0.3473389772626151
Iteration 15 err= 0.24735109800044114
Iteration 16 err= 0.10361233537093052
Iteration 17 err= 0.06641958990762059
Iteration 18 err= 0.03727265361879442
Iteration 19 err= 0.016176978345660543
Iteration 20 err= 0.012252517138652217
Iteration 21 err= 0.005913601743073285
Iteration 22 err= 0.0056326003413210484
Iteration 23 err= 0.0027256640245714296
Iteration 24 err= 0.0022963642507545672
Iteration 25 err= 0.001198248430956027
Iteration 26 err= 0.0006304630

WebGuiWidget(value={'ngsolve_version': '6.2.2105-264-g4a22558b6', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

for convenience of the reader, we started from this version of the usual preconditioned cg-iteration:

In [5]:
def CG(A,b,pre, maxit=200, tol=1e-8):
    x = b.CreateVector()
    w = b.CreateVector()
    r = b.CreateVector()
    p = b.CreateVector()
    ap = b.CreateVector()

    r.data = b
    p.data = pre*r
    wrn = InnerProduct(r,p)
    err0 = sqrt(wrn)
    x[:] = 0
    
    for it in range(maxit):
        ap.data = A * p

        pap = InnerProduct(p, ap)
        wr = wrn
        alpha = wr / pap
        
        x.data += alpha * p
        r.data -= alpha * ap
        w.data = pre*r
        
        wrn = InnerProduct(w, r)
        err = sqrt(wrn)
        print ("Iteration",it,"err=",err)
        if err < tol * err0: break
            
        beta = wrn / wr
        
        p *= beta
        p.data += w 
        
    return x